In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime as dt

In [2]:
md = "ADJ_CLOSE.csv"
dsi_range = ["1991-01-01", "2020-12-31"] # DSI is calculated from 1991 to 2020
sim_range = ["1990-01-01", "2010-12-31"] # simulation uses data from 1990 to 2010
val_range = ["2010-01-01", "2020-12-31"] # validation uses data from 2010 t0 2020

In [3]:
df = pd.read_csv(md, usecols = ['Date','vix'])
df

,Date,vix
0,1990-01-02,17.24
1,1990-01-03,18.19
2,1990-01-04,19.22
3,1990-01-05,20.11
4,1990-01-08,20.26
...,...,...
7806,2020-12-23,23.31
7807,2020-12-24,21.53
7808,2020-12-28,21.70
7809,2020-12-29,23.08


In [4]:
df["Date"] = pd.to_datetime(df["Date"])

In [5]:
df['Year_Month'] = df["Date"].dt.to_period('M')
df = df.set_index("Year_Month")

In [6]:
df

,Date,vix
Year_Month,,
1990-01,1990-01-02,17.24
1990-01,1990-01-03,18.19
1990-01,1990-01-04,19.22
1990-01,1990-01-05,20.11
1990-01,1990-01-08,20.26
...,...,...
2020-12,2020-12-23,23.31
2020-12,2020-12-24,21.53
2020-12,2020-12-28,21.70


In [7]:
#get rolling median vix for the past 12 months
vix_median = df.groupby([df.index.year]).median().shift(periods=1,fill_value=0)

In [8]:
#convert 
vix_Dict = dict(vix_median.iloc[:,-1])

In [9]:
df["HVM-12"] = df.index.year.map(vix_Dict)

In [10]:
conditions = [(df["vix"] > df["HVM-12"]),(df["vix"] < df["HVM-12"])]
val = ["Fearful","Greedy"]

df["DSI"] = np.select(conditions,val)

In [11]:
df = df["1991":]
df

,Date,vix,HVM-12,DSI
Year_Month,,,,
1991-01,1991-01-02,26.62,22.570,Fearful
1991-01,1991-01-03,27.93,22.570,Fearful
1991-01,1991-01-04,27.19,22.570,Fearful
1991-01,1991-01-07,28.95,22.570,Fearful
1991-01,1991-01-08,30.38,22.570,Fearful
...,...,...,...,...
2020-12,2020-12-23,23.31,14.865,Fearful
2020-12,2020-12-24,21.53,14.865,Fearful
2020-12,2020-12-28,21.70,14.865,Fearful


In [12]:
ftd = df.groupby(df['Date'].dt.to_period('M')).min()
ftd = ftd[["Date"]].rename(columns={"Date":"FTD"})
ftd

,FTD
Date,
1991-01,1991-01-02
1991-02,1991-02-01
1991-03,1991-03-01
1991-04,1991-04-01
1991-05,1991-05-01
...,...
2020-08,2020-08-03
2020-09,2020-09-01
2020-10,2020-10-01


In [13]:
ltd = df.groupby(df['Date'].dt.to_period('M')).max()
ltd = ltd[["Date"]].rename(columns={"Date":"LTD"})
ltd

,LTD
Date,
1991-01,1991-01-31
1991-02,1991-02-28
1991-03,1991-03-28
1991-04,1991-04-30
1991-05,1991-05-31
...,...
2020-08,2020-08-31
2020-09,2020-09-30
2020-10,2020-10-30


In [14]:
interim_df = df.groupby(["Year_Month","DSI"]).size().to_frame()
interim_df2 = interim_df.reset_index()
months = interim_df2["Year_Month"].unique()
interim_df["MSI"] = 0

In [15]:
iterr=0
for i in months:
    iterr+=1
    if interim_df.loc[i,0].index[0] == "Fearful" and interim_df.loc[i,0].index[-1]=="Greedy":
        #get number of fearful in each month
        x = interim_df.loc[i,0][0]
        #get number of greedy 
        y = interim_df.loc[i,0][1]
        
        if x>y:
            interim_df.loc[i,["MSI"]] = "Fearful"
        else:
            interim_df.loc[i,["MSI"]] = "Greedy"
        
    elif interim_df.loc[i,0].index[0] == "Fearful" and interim_df.loc[i,0].index[-1]=="Fearful":
            #only fearful
        interim_df.loc[i,["MSI"]] = "Fearful"
    
    else:
        #only greedy
        interim_df.loc[i,["MSI"]] = "Greedy"

In [16]:
interim_df3 = interim_df.reset_index()
interim_df3 = interim_df3[["Year_Month","MSI"]].groupby(["Year_Month"]).first()

In [17]:
df2 = pd.concat([ltd,ftd,interim_df3],axis=1)
df2.set_index("LTD",inplace=True)
df2["FTD"] = df2[["FTD"]].shift(-1)
df2

,FTD,MSI
LTD,,
1991-01-31,1991-02-01,Fearful
1991-02-28,1991-03-01,Greedy
1991-03-28,1991-04-01,Greedy
1991-04-30,1991-05-01,Greedy
1991-05-31,1991-06-03,Greedy
...,...,...
2020-08-31,2020-09-01,Fearful
2020-09-30,2020-10-01,Fearful
2020-10-30,2020-11-02,Fearful


In [18]:
p = df.set_index("Date")
p["MSI"] = 0
p["TS"] = 0
p

,vix,HVM-12,DSI,MSI,TS
Date,,,,,
1991-01-02,26.62,22.570,Fearful,0,0
1991-01-03,27.93,22.570,Fearful,0,0
1991-01-04,27.19,22.570,Fearful,0,0
1991-01-07,28.95,22.570,Fearful,0,0
1991-01-08,30.38,22.570,Fearful,0,0
...,...,...,...,...,...
2020-12-23,23.31,14.865,Fearful,0,0
2020-12-24,21.53,14.865,Fearful,0,0
2020-12-28,21.70,14.865,Fearful,0,0


In [19]:
#input Greedy/Fearful on last trading days
def inputMSI(p,q):
    for i in q.index:
        p.loc[i,"MSI"] = q.loc[i,"MSI"]
    return p

In [20]:
inputMSI(p,df2)

,vix,HVM-12,DSI,MSI,TS
Date,,,,,
1991-01-02,26.62,22.570,Fearful,0,0
1991-01-03,27.93,22.570,Fearful,0,0
1991-01-04,27.19,22.570,Fearful,0,0
1991-01-07,28.95,22.570,Fearful,0,0
1991-01-08,30.38,22.570,Fearful,0,0
...,...,...,...,...,...
2020-12-23,23.31,14.865,Fearful,0,0
2020-12-24,21.53,14.865,Fearful,0,0
2020-12-28,21.70,14.865,Fearful,0,0


In [21]:
#Add buy/sell signal
def signal(x,y):
    for i in x.index:
        if i < y.index[-1]:
            if y.loc[i,"MSI"]=="Fearful":
                y.loc[(x.loc[i,"FTD"]),"TS"] = "Buy"
            else:
                y.loc[(x.loc[i,"FTD"]),"TS"] = "Sell"
                
    return y

In [22]:
TS = signal(df2,p)

In [23]:
TS = TS.loc[TS.TS!=0]
TS

,vix,HVM-12,DSI,MSI,TS
Date,,,,,
1991-02-01,21.04,22.570,Greedy,0,Buy
1991-03-01,21.23,22.570,Greedy,0,Sell
1991-04-01,17.42,22.570,Greedy,0,Sell
1991-05-01,17.77,22.570,Greedy,0,Sell
1991-06-03,16.87,22.570,Greedy,0,Sell
...,...,...,...,...,...
2020-08-03,24.28,14.865,Fearful,0,Buy
2020-09-01,26.12,14.865,Fearful,0,Buy
2020-10-01,26.70,14.865,Fearful,0,Buy


In [24]:
TS.loc["1997-06-02"]

vix         20.85
HVM-12     16.245
DSI       Fearful
MSI             0
TS            Buy
Name: 1997-06-02 00:00:00, dtype: object

In [25]:
stocks = pd.read_csv(md).drop("vix",axis=1)
stocks["Date"] = pd.to_datetime(stocks["Date"])
stocks = stocks.set_index("Date")
stocks = stocks[sim_range[0]:sim_range[1]]

In [26]:
stocks

,s_0000,s_0001,s_0002,s_0003,s_0004,s_0005,s_0006,s_0007,s_0008,s_0009,...,s_2579,s_2580,s_2581,s_2582,s_2583,s_2584,s_2585,s_2586,s_2587,s_2588
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,13.389421,NaN,NaN,NaN,NaN,NaN,0.266812,NaN,NaN,...,NaN,1.950358,NaN,7.253997,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,13.588601,NaN,NaN,NaN,NaN,NaN,0.268603,NaN,NaN,...,NaN,1.985185,NaN,7.253997,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,13.610730,NaN,NaN,NaN,NaN,NaN,0.269499,NaN,NaN,...,NaN,1.985185,NaN,7.188052,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,13.477942,NaN,NaN,NaN,NaN,NaN,0.270394,NaN,NaN,...,NaN,1.985185,NaN,7.188052,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,13.477942,NaN,NaN,NaN,NaN,NaN,0.272185,NaN,NaN,...,NaN,1.985185,NaN,7.385889,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-27,27.404488,34.108845,9.304858,NaN,NaN,7.825823,63.881012,9.956269,NaN,57.250000,...,47.759998,20.831934,4.83,4.490000,NaN,NaN,NaN,NaN,NaN,28.830000
2010-12-28,27.136467,34.153625,9.323714,NaN,NaN,7.769111,63.717537,9.980500,NaN,56.500000,...,45.119999,21.143118,4.71,4.260000,NaN,NaN,NaN,NaN,NaN,28.650000
2010-12-29,27.254137,33.884895,9.559400,NaN,NaN,7.790717,63.909893,9.974977,NaN,56.810001,...,44.480000,20.978882,4.75,4.280000,NaN,NaN,NaN,NaN,NaN,28.209999


In [27]:
ticker = stocks.columns

In [28]:
stocks = stocks.loc[stocks.index.isin(TS.index)].join(TS)

In [29]:
stocks

,s_0000,s_0001,s_0002,s_0003,s_0004,s_0005,s_0006,s_0007,s_0008,s_0009,...,s_2584,s_2585,s_2586,s_2587,s_2588,vix,HVM-12,DSI,MSI,TS
Date,,,,,,,,,,,,,,,,,,,,,
1991-02-01,NaN,12.312566,NaN,NaN,NaN,NaN,NaN,0.404282,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,21.04,22.57,Greedy,0,Buy
1991-03-01,NaN,12.406919,NaN,NaN,NaN,NaN,NaN,0.419667,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,21.23,22.57,Greedy,0,Sell
1991-04-01,NaN,12.171042,NaN,NaN,NaN,NaN,NaN,0.497787,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,17.42,22.57,Greedy,0,Sell
1991-05-01,NaN,13.297199,NaN,NaN,NaN,NaN,NaN,0.343364,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,17.77,22.57,Greedy,0,Sell
1991-06-03,NaN,13.605876,NaN,NaN,NaN,NaN,NaN,0.358813,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,16.87,22.57,Greedy,0,Sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-08-02,18.840595,26.098185,10.030772,NaN,NaN,6.848853,51.289085,8.029595,NaN,59.250000,...,NaN,NaN,NaN,NaN,18.610001,22.01,28.57,Greedy,0,Sell
2010-09-01,18.494114,23.507025,9.012609,NaN,NaN,6.039004,53.256954,7.676337,NaN,45.369999,...,NaN,NaN,NaN,NaN,16.000000,23.89,28.57,Greedy,0,Sell
2010-10-01,22.050423,27.328032,8.758069,NaN,NaN,6.280349,56.405876,8.663441,NaN,50.110001,...,NaN,NaN,NaN,NaN,21.170000,22.50,28.57,Greedy,0,Sell


In [32]:
def simulation(capital:float,stocks,ticker):
    x=[]
    
    for i in ticker:
        row=0
        stock_qty = 0
        cost = 0
        running_bal = capital
        PL = 0
        
        for date in stocks.index:
            row+=1
            if np.isnan(stocks.loc[date,i]):
                pass
            else:
                price = stocks.loc[date,i]
                
                if stocks.loc[date,"TS"] == "Buy" and (stocks.loc[date:,"TS"].isin(["Sell"]).any()) and stock_qty == 0:
                    stock_qty = math.floor(running_bal/price)
                    cost = round(stock_qty*price,3)
                    running_bal = round(running_bal - cost,3)
            
                elif stocks.loc[date,"TS"] == "Sell" and stock_qty!=0:
                    value = stock_qty*price
                    running_bal = round(running_bal + value,3)
                    stock_qty = 0
        
        
            #print("row:",row,"qty:",stock_qty,"cost:",cost,"bal:",running_bal,stocks.loc[date,"TS"])
        PL = running_bal - capital
        x.append(PL)
    
    return x

In [33]:
x = simulation(10000,stocks,ticker)

In [34]:
strategyDF = pd.DataFrame(x,columns=["P&L"])
strategyDF = strategyDF.sort_values("P&L",ascending=False)

In [35]:
for i in range(5):
    endVal = strategyDF.iloc[i,0]+10000
    PL = endVal - 10000
    returns = ((endVal/10000)**(1/20) - 1)*100
    print(f"Stock #{(strategyDF.index[i]):<5} P&L: ${PL:,.2f} Annual Return:{returns:.2f}%")

Stock #145   P&L: $635,935.25 Annual Return:23.17%
Stock #1555  P&L: $301,095.53 Annual Return:18.75%
Stock #2052  P&L: $221,923.31 Annual Return:17.02%
Stock #2206  P&L: $218,792.72 Annual Return:16.94%
Stock #1862  P&L: $196,422.42 Annual Return:16.34%


In [36]:
Top5 = strategyDF.index[:5]

def rename(lst):
    a=[]
    for i in lst:
        x=str(i)
        if len(x)==2:
            x="s_00"+x
            a.append(x)
        elif len(x)==3:
            x="s_0"+x
            a.append(x)
        else:
            x="s_"+x
            a.append(x)
    
    return a


Top5 = rename(Top5)

In [37]:
val_df = pd.read_csv(md).drop("vix",axis=1)
val_df["Date"] = pd.to_datetime(val_df["Date"])
val_df = val_df.set_index("Date")
val_df = val_df[val_range[0]:val_range[1]]
val_df

,s_0000,s_0001,s_0002,s_0003,s_0004,s_0005,s_0006,s_0007,s_0008,s_0009,...,s_2579,s_2580,s_2581,s_2582,s_2583,s_2584,s_2585,s_2586,s_2587,s_2588
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,20.461840,36.938263,4.496877,NaN,NaN,5.318474,38.662930,6.562588,NaN,38.200001,...,NaN,11.501429,2.95,1.78,NaN,NaN,NaN,NaN,NaN,12.360000
2010-01-05,20.239578,35.784649,5.005958,NaN,NaN,5.164161,38.433144,6.573934,NaN,40.310001,...,NaN,11.906957,3.05,1.70,NaN,NaN,NaN,NaN,NaN,12.290000
2010-01-06,20.167669,37.648201,4.798554,NaN,NaN,4.959297,38.768265,6.469369,NaN,38.630001,...,NaN,12.942341,3.08,1.70,NaN,NaN,NaN,NaN,NaN,12.680000
2010-01-07,20.141516,36.849533,4.939965,NaN,NaN,5.142877,38.758690,6.457407,NaN,38.950001,...,NaN,14.391890,3.04,2.22,NaN,NaN,NaN,NaN,NaN,14.660000
2010-01-08,20.134979,37.759125,4.845690,NaN,NaN,5.204072,38.911869,6.500339,NaN,39.270000,...,NaN,14.158920,3.10,2.12,NaN,NaN,NaN,NaN,NaN,14.730000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,116.631310,22.171579,15.890000,16.577030,9.00,65.023491,157.495850,130.347580,27.481012,55.750000,...,19.870001,42.675430,2.90,8.85,9.93,NaN,0.226,207.470001,158.974014,36.650002
2020-12-24,116.641243,21.912146,15.660000,16.606722,8.77,65.292725,158.870193,131.352829,27.813406,56.139999,...,20.180000,42.508686,2.88,8.81,9.91,NaN,0.229,205.270004,159.839264,36.009998
2020-12-28,117.158287,22.191536,16.059999,16.200956,8.93,66.429459,157.011383,136.050766,28.272888,54.830002,...,19.959999,42.528305,2.69,8.76,9.73,NaN,0.251,199.369995,161.500122,36.709999


In [38]:
sim = val_df.loc[:,Top5]

In [39]:
sim = sim.loc[sim.index.isin(TS.index)].join(TS)

In [40]:
sim

,s_0145,s_1555,s_2052,s_2206,s_1862,vix,HVM-12,DSI,MSI,TS
Date,,,,,,,,,,
2010-01-04,133.899994,96.160004,17.086140,13.355339,16.751551,20.04,28.570,Greedy,0,Sell
2010-02-01,118.870003,94.370003,15.928634,11.931488,14.612470,22.59,28.570,Greedy,0,Sell
2010-03-01,124.540001,90.669998,15.818738,14.121334,17.484974,19.26,28.570,Greedy,0,Sell
2010-04-01,131.809998,85.470001,16.139465,13.959122,18.652164,17.47,28.570,Greedy,0,Sell
2010-05-03,137.490005,76.089996,16.954233,15.716405,17.425470,20.19,28.570,Greedy,0,Sell
...,...,...,...,...,...,...,...,...,...,...
2020-08-03,3111.889893,123.660004,32.763863,144.897614,60.196865,24.28,14.865,Fearful,0,Buy
2020-09-01,3499.120117,148.820007,35.367496,144.670258,56.493301,26.12,14.865,Fearful,0,Buy
2020-10-01,3221.260010,148.960007,37.274185,150.241348,54.852787,26.70,14.865,Fearful,0,Buy


In [41]:
validation = simulation(10000,sim,Top5)

In [42]:
for i,v in enumerate(validation):
    endVal = v+10000
    returns = ((endVal/10000)**(1/10) - 1)*100
    print(f"Stock #{(sim.columns[i]):<5} P&L: ${v:>10,.2f} Annual Return:{returns:>5.2f}%")

Stock #s_0145 P&L: $ 12,917.06 Annual Return: 8.65%
Stock #s_1555 P&L: $  8,886.52 Annual Return: 6.57%
Stock #s_2052 P&L: $   -408.52 Annual Return:-0.42%
Stock #s_2206 P&L: $ 11,683.95 Annual Return: 8.05%
Stock #s_1862 P&L: $  9,517.56 Annual Return: 6.92%
